In [14]:
import torch
def t(value):
    return torch.Tensor([value]).requires_grad_(True)

a=t(343.212)
b=t(543)
c=t(5.3)
d=t(54.1)

erro = (5+a)*b+(c*d/a)

erro.backward()
c.grad

tensor([0.1576])

In [1]:
import math
class Drv:
    def __init__(self,num):
        self.num=num
        self.grad=0
        self.ant = ()
        self._backward = lambda: None

    def __str__(self):
        return f"num: {self.num} grad: {self.grad} anteriores {[a.num for a in self.ant]}"
    
    def __add__(self,outro):
        novo = Drv(self.num+outro.num)
        novo.ant = (self,outro)
        def drv_adicao():
            self.grad+=1*novo.grad
            outro.grad+=1*novo.grad
        novo._backward = drv_adicao
        return novo
    
    def __mul__(self,outro):
        novo = Drv(self.num*outro.num)
        novo.ant = (self,outro)
        def drv_mult():
            self.grad+=outro.num*novo.grad
            outro.grad+=self.num*novo.grad
        novo._backward = drv_mult
        return novo
    
    def sig(self):
        x = self.num
        
        if(x>15):
            sigx=0.99
        elif(x<-15):
            sigx=0.01
        else:
            sigx = 1/(1+math.exp(-x))

        novo = Drv(sigx)
        novo.ant=(self,)
        def drv_sig():
            self.grad+= sigx*(1-sigx)*novo.grad
        novo._backward=drv_sig
        return novo
    

    def setar_derivadas_em_todos_os_anteriores(self):
        ordem_topo=[]
        visitados=set()
        def construir_ordem_topo(no:Drv):
            if no not in visitados:
                visitados.add(no)
                for var in no.ant:
                    construir_ordem_topo(var)
                ordem_topo.append(no)
        construir_ordem_topo(self)

        self.grad=1
        for dr in reversed(ordem_topo):
            dr._backward()
    
    

In [ ]:
import random
class No():
    def __init__(self, qtd_en):
        self.pesos =  [Drv(random.randrange(-100,100)/100) for _ in range(qtd_en)]
        self.vies = Drv(random.randrange(-100,100)/100)
    def __call__(self,entradas):
        if(len(self.pesos)==1):
            entradas=[entradas]

        saida= Drv(0)
        saida= self.vies+saida
        for i,p in enumerate(self.pesos):
            saida+= p*entradas[i]
        return saida.sig()
    
    def params(self):
        return self.pesos+[self.vies]

class Camada():
    def __init__(self,qtd_entradas,qtd_nos):
        self.nos = [No(qtd_entradas) for _ in range(qtd_nos)]
    def __call__(self,entradas):
        return [no(entradas) for no in self.nos]
    def params(self):
        parametros = []
        for no in self.nos:
            parametros+=no.params()
        return parametros

class MLP():
    def __init__(self,entradas,neuronios_em_camadas):
        self.camadas=[]
        self.camadas.append(Camada(entradas,neuronios_em_camadas[0]))
        for i,n in enumerate(neuronios_em_camadas[1:]):
            self.camadas.append(Camada(neuronios_em_camadas[i],n))

    def __call__(self,entradas):
        saida = entradas
        for c in self.camadas:
            saida=c(saida)
        return saida
    def params(self):
        parametros=[]
        for c in self.camadas:
            parametros+=c.params()
        return parametros
    
    
x = [
    [Drv(-1)],
    [Drv(0)],
    [Drv(1.0)]
]
y = [Drv(2), Drv(1), Drv(2.0)]

rede = MLP(1,[50,30,1])

tx_ap = 0.1
for i in range(500):
    erro_acumulado =Drv(0)
    for e,s in zip(x,y):
        y_f = rede(e)[0]
        menos_y=Drv(-1)*s
        erro = (y_f+menos_y)
        erro_quadrado=erro*erro
        erro_acumulado+=erro_quadrado
    print(f"erro_acumulado {erro_acumulado}")
    erro_acumulado.setar_derivadas_em_todos_os_anteriores()
    for p in rede.params():
        p.num += -p.grad * tx_ap #Adam()
        p.grad=0
        

erro_acumulado num: 1.7043431064217094 grad: 0 anteriores [1.7009706401278308, 0.003372466293878609]
erro_acumulado num: 1.5873438926175347 grad: 0 anteriores [1.5808612605258336, 0.006482632091701093]
erro_acumulado num: 1.3907089198500044 grad: 0 anteriores [1.3760503234833812, 0.014658596366623285]
erro_acumulado num: 1.097682122250844 grad: 0 anteriores [1.0592343482428832, 0.03844777400796066]
erro_acumulado num: 0.8344051714943271 grad: 0 anteriores [0.7404752275997064, 0.09392994389462074]
erro_acumulado num: 0.737267421062787 grad: 0 anteriores [0.5865418028619458, 0.15072561820084124]
erro_acumulado num: 0.7062414657319837 grad: 0 anteriores [0.5326420322756438, 0.17359943345633985]
erro_acumulado num: 0.6852641834637627 grad: 0 anteriores [0.5050828146475659, 0.1801813688161968]
erro_acumulado num: 0.6676643963811224 grad: 0 anteriores [0.4853859071400414, 0.1822784892410811]
erro_acumulado num: 0.6524405976194201 grad: 0 anteriores [0.4690931675323257, 0.18334743008709434]
e